## EnergyPlus Python API Demo

In [8]:
import sys
import os
ep_install_path = "/usr/local/EnergyPlus-23-2-0/"
sys.path.append(ep_install_path)
cwd = os.getcwd()

output_dir = '/home/arschall/Programming/ASSET/EnergyPlusPython/EPcomparison_pysimple'
epw_dir = cwd + '/EPcomparison_pysimple/1992_historical_detroit_H.epw'
idf_dir = cwd + '/EPcomparison_pysimple/in_DT92H.idf'

from pyenergyplus.api import EnergyPlusAPI

In [9]:
api = EnergyPlusAPI()
print("EnergyPlus Version: " + str(api.functional.ep_version()))

EnergyPlus Version: 23.2.0-7636e6b3e9


### Functional API Demo
The functional API allows for interfacing with static information through the EnergyPlus binary. To interact with a property, an object of that property must be instantiated. Once an object exists, it can be altered through class methods.

In [3]:
api = EnergyPlusAPI()

state = api.state_manager.new_state() # TODO: purpose of state_manager and state?

glycol = api.functional.glycol(state, u"water")
cp = glycol.specific_heat(state, 35.0)
rho = glycol.density(state, 15.0)
glycol.delete(state)

refrigerant = api.functional.refrigerant(state, "steam")
satPress = refrigerant.saturation_pressure(state, 100.0)
refrigerant.delete(state)

### Runtime API Demo
The runtime API allows users to initiate and interact with running simulations. The user can create callback functions that are handled throughout simulation runtime 

In [10]:
api = EnergyPlusAPI()
state = api.state_manager.new_state()
progressValue = 0

def progress_handler(progress: int) -> None:
    global progressValue
    progressValue = progress
    if 49 < progress < 51:
        print("------Halfway through simulation------")
        sys.stdout.flush()

api.runtime.callback_progress(state, progress_handler)

v = api.runtime.run_energyplus(state, ['-d', output_dir, '-w', epw_dir, idf_dir])
if v != 0:
    print("EnergyPlus Simulation Failed")
    sys.exit(1)
assert(progressValue == 100)

EnergyPlus Starting
EnergyPlus, Version 23.2.0-7636e6b3e9, YMD=2024.01.10 10:45
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Starting Simulation at 01/01/2007 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=01/21/2007
Continuing Simulation at 01/21/2007 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=02/10/2007
Continuing Simulation at 02/10/2007 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=03/02/2007
Continuing Simulation at 03/02/2007 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=03/22/2007
Continuing Simulation at 03/22/2007 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=04/11/2007
Continuing Simulation at 04/11/2007 for RUN PERIOD 1
Updating Shadowing Calculations, Start Date=05/01/2007
Continuing Simulation at 05/01/2007 for RUN PE

EnergyPlus Completed Successfully.


### Data Exchange API Demo
The runtime API allows users to interact with simulation data, categorized into:
- __Variables:__ Time series output variables from simulation. Must be requested to be tracked by adding `Output:Variable` in the input file. Variables accessed with a `get` method.
- __Meters:__  Collection of variables from simulation. Do not need to be explicitly requested. Meters accessed with a `get` method.
- __Internal Variables:__ "Static" variables that do not change over a simulation period. Internal Variables accessed with `get` method.
- __Simulation Parameters:__ Simulation parameters that vary throughout the simulation period. Examples include current simulation day of week or year, or current simulation hour. Available through direct function calls.
- __Actuators:__ Entry point for users to view and modify runtime variables during simulation. Though there are restrictions to limit actuation to respect environmental boundaries, users can access, perform calculations upon, and update Data Exchange variables.